In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from scipy.stats import ks_2samp

# League of Legends Dataset Proof of Concept

# Multitype Columns Issue
- When loading csv file,there are some multi type columns, I don't know if that's an issue because when I look into it, it looks like the only multitype columns comes from missing data.
- From first viewing, I can assume that most of the missingness is missing by design. Some of the player names are missing because there are rows that represent the statistics of a whole team in a game (shown later).

In [2]:
league = pd.read_csv("./2022_LoL_esports_match_data_from_OraclesElixir_20221023.csv")
league.head()

C:\Users\costi\AppData\Local\Temp\ipykernel_11684\2508018919.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  league = pd.read_csv("./2022_LoL_esports_match_data_from_OraclesElixir_20221023.csv")


,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0


In [3]:
my_cols = league.columns[(league.applymap(type).apply(pd.value_counts).fillna(0) > 0).sum() > 1].values
my_cols

array(['url', 'split', 'playername', 'playerid', 'teamname', 'teamid',
       'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5'], dtype=object)

In [4]:
league.applymap(type).apply(pd.value_counts).fillna(0)[my_cols]

,url,split,playername,playerid,teamname,teamid,champion,ban1,ban2,ban3,ban4,ban5
<class 'str'>,21480.0,96924.0,118100.0,116495.0,141666.0,140064.0,118100.0,139944.0,140040.0,139710.0,139860.0,139554.0
<class 'float'>,120240.0,44796.0,23620.0,25225.0,54.0,1656.0,23620.0,1776.0,1680.0,2010.0,1860.0,2166.0
<class 'int'>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Investigating Data by Game
- For each game there are 5 players on each team, thus we suspect that each unique gameid has 10 corresponding rows.
    - However there are 12 per game, this is because there are two additional rows for aggregate team results which is great for students who want to do data by teams.

In [5]:
# We want to run count on a non-nan column
league.columns[(~league.isna()).all()]

Index(['gameid', 'datacompleteness', 'league', 'year', 'playoffs', 'date',
       'game', 'participantid', 'side', 'position', 'gamelength', 'result',
       'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'team kpm',
       'ckpm', 'totalgold'],
      dtype='object')

In [6]:
# 12 rows per game
league.groupby("gameid")["participantid"].count().unique()

array([12], dtype=int64)

In [7]:
# NaN playername rows seem to contain the aggregate info for a team, note the unique "team" position value in contast
# To the typical 5 player roles (top, jungle, mid, bottom, support)
selected_game = league["gameid"].iloc[0]
game_info = league.loc[league["gameid"] == selected_game,["playername","teamname","kills","teamkills","position","result"]]
game_info

,playername,teamname,kills,teamkills,position,result
0,Soboro,Fredit BRION Challengers,2,9,top,0
1,Raptor,Fredit BRION Challengers,2,9,jng,0
2,Feisty,Fredit BRION Challengers,2,9,mid,0
3,Gamin,Fredit BRION Challengers,2,9,bot,0
4,Loopy,Fredit BRION Challengers,1,9,sup,0
5,DnDn,Nongshim RedForce Challengers,1,19,top,1
6,Sylvie,Nongshim RedForce Challengers,4,19,jng,1
7,FIESTA,Nongshim RedForce Challengers,6,19,mid,1
8,vital,Nongshim RedForce Challengers,8,19,bot,1
9,Blessing,Nongshim RedForce Challengers,0,19,sup,1


In [8]:
# Note some columns that are MD from looking at the "position" value. There is a mapping of team rows to missingness in the
# following columns which makes sence as they are player only statistics.
league.columns[league[league["position"] == "team"].isna().all()]

Index(['playername', 'playerid', 'champion', 'firstbloodkill',
       'firstbloodassist', 'firstbloodvictim', 'damageshare',
       'earnedgoldshare', 'total cs'],
      dtype='object')

## Trying to find MAR and NMAR data columns
- Note that this dataset has a `datacompleteness` column to represent rows that have missingness, we may tell students to ignore the column when determining NMAR and MAR because the information that this column provides is just that the data is missing thus being MAR with this column feels pretty redundant.

In [9]:
def assess_missingness(data,focus_col,compare_col,stat="tvd",n_repetitions=1000):
    def tvd(data,missing_col,compare_col):
        pivoted = (
            shuffled
            .pivot_table(index=missing_col, columns=compare_col, aggfunc='size')
            .apply(lambda x: x / x.sum(), axis=1)
        )

        tvd = pivoted.diff().iloc[-1].abs().sum() / 2
        return tvd
    
    def ks_test(data,missing_col,compare_col):
        missing_data = data.loc[data[missing_col],compare_col]
        nonmissing_data = data.loc[~data[missing_col],compare_col]
        return ks_2samp(missing_data, nonmissing_data).pvalue
    
    shuffled = data.copy()
    missing_col = focus_col+'_missing'
    shuffled[missing_col] = shuffled[focus_col].isna()
    
    
    if stat == "ks":
        return ks_test(shuffled,missing_col,compare_col)
    
    if stat == "tvd":
        obs_tvd = tvd(data,missing_col,compare_col)
        tvds = []
        for _ in range(n_repetitions):

            # Shuffling genders and assigning back to the DataFrame
            shuffled[compare_col] = np.random.permutation(shuffled[compare_col])
            tvds.append(tvd(shuffled,missing_col,compare_col))
        pval = np.mean(np.array(tvds) >= obs_tvd)
        return pval

In [10]:
# Missingness of the split is dependent on the league the game was help
# This makes sense because some leagues may not define their tourneys by splits
assess_missingness(league,"split","league")

0.0

In [11]:
# Missingness of the split is not dependent on the number of dragons a player or team killed 
assess_missingness(league,"split","dragons",stat="ks")

0.9999744015450113

## Potential Lines of inquiry for Project 3
- Looking at [tier one professional leagues](https://en.wikipedia.org/wiki/List_of_League_of_Legends_leagues_and_tournaments), which league has the most action packed games and for that league, is it's level of action significantly different than the others?
    - We will look at average kills per minute (kpm) and use a ks stat
- Who "carries" their team more often, ADCs (Bot lanes) or Mid laners?
    - We will look at gold per minute (gpm)
- Is the average win rate of my favorite champion (Talon) different or similar to the average win rate of all other champions in the dataset?
     - We will obviously look at winrates by averaging the results column

In [12]:
def permutation_test(data,focus_col,focus_group,compare_col,stat="tvd",n_repetitions=1000):
    def tvd(data,AB_col,compare_col):
        pivoted = (
            data
            .pivot_table(index=AB_col, columns=compare_col, aggfunc='size')
            .apply(lambda x: x / x.sum(), axis=1)
        )

        tvd = pivoted.diff().iloc[-1].abs().sum() / 2
        return tvd
    
    def ks_test(data,AB_col,compare_col):
        target_data = data.loc[data[AB_col],compare_col]
        nontarget_data = data.loc[~data[AB_col],compare_col]
        return ks_2samp(target_data, nontarget_data).pvalue
    
    def diff_means(data,AB_col,compare_col):
        target_data = data.loc[data[AB_col],compare_col]
        nontarget_data = data.loc[~data[AB_col],compare_col]
        return target_data.mean() - nontarget_data.mean()
    
    shuffled = data.copy()
    AB_col = focus_col+'_istarget'
    shuffled[AB_col] = shuffled[focus_col] == focus_group
    
    
    if stat == "ks":
        return ks_test(shuffled,AB_col,compare_col)
    
    if stat == "diff":
        obs_diff = diff_means(shuffled,AB_col,compare_col)
        diffs = []
        for _ in range(n_repetitions):
            shuffled[compare_col] = np.random.permutation(shuffled[compare_col])
            diffs.append(diff_means(shuffled,AB_col,compare_col))
        pval = np.mean(np.array(diffs) >= obs_diff)
        return pval
    
    if stat == "tvd":
        obs_tvd = tvd(shuffled,AB_col,compare_col)
        tvds = []
        for _ in range(n_repetitions):
            shuffled[compare_col] = np.random.permutation(shuffled[compare_col])
            tvds.append(tvd(shuffled,AB_col,compare_col))
        pval = np.mean(np.array(tvds) >= obs_tvd)
        return pval

## Question 1
Note to get the average kpm in a game we can sum the kills per minute for both teams in a game

In [13]:
#t1_leagues are defined here: https://en.wikipedia.org/wiki/List_of_League_of_Legends_leagues_and_tournaments
t1_leagues = ["LCK","LCS","LEC","LPL","PCS","VCS","CBLOL","LCO","LCL","LJL","LLA","TCL"]

In [14]:
# Get all games in upper division leagues and only get the team data to avoid repeats
t1_matches = league[league["league"].isin(t1_leagues)]
t1_matches = t1_matches[t1_matches["position"] == "team"]
t1_matches.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
34,8401-8401_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=8401,LPL,2022,Spring,0,2022-01-10 09:24:26,1,12.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,8401-8401_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=8401,LPL,2022,Spring,0,2022-01-10 09:24:26,1,12.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,8401-8401_game_2,partial,https://lpl.qq.com/es/stats.shtml?bmid=8401,LPL,2022,Spring,0,2022-01-10 10:09:22,2,12.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,8401-8401_game_2,partial,https://lpl.qq.com/es/stats.shtml?bmid=8401,LPL,2022,Spring,0,2022-01-10 10:09:22,2,12.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,8402-8402_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=8402,LPL,2022,Spring,0,2022-01-10 11:26:11,1,12.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Aggregate rows to one row per game containing that games kpm
kpm_per_game = t1_matches.groupby("gameid").agg({"league":lambda s: s.unique()[0],"team kpm":"sum"})
kpm_per_game.head()

,league,team kpm
gameid,,
8401-8401_game_1,LPL,0.8351
8401-8401_game_2,LPL,1.2465
8402-8402_game_1,LPL,0.6339
8402-8402_game_2,LPL,0.5020
8402-8402_game_3,LPL,0.8526


In [16]:
# VCS looks like the highest KPM, but is it significantly different from the average KPM of all other leagues?
kpm_per_game.groupby("league")["team kpm"].mean().sort_values()

league
LCK      0.700287
LCS      0.733368
LCL      0.765912
LJL      0.778505
LLA      0.803713
LEC      0.804729
PCS      0.833358
LPL      0.839326
TCL      0.853934
CBLOL    0.874237
LCO      0.959596
VCS      1.053568
Name: team kpm, dtype: float64

In [17]:
permutation_test(kpm_per_game,"league","VCS","team kpm","diff")

0.0

So the results shows that the VCS league has a statitically significantly higher kpm that other games which shows (if we use kpm as a measure of how action filled a game is) that the VCS league is the most action packed compared to the other leagues.

## Potential Lines of inquiry for Project 5
- Given information about a team, will that team win their match (classification)?
    - To make this easier, we can use data 15 minutes into a match.
    - By adding information about the enemy team, the model will likely be more accurate.
- Given information about a team, how much will their KPM be in a match (regression)?